# Fx Forward Pricing

This notebook will run through the following business use cases :
* [Pricing an Fx Forward using a supplied vendor model.](#pricing_bond)
* [Valuing a portfolio with an Fx Forward position.](#pricing_bond_portfolio)
* [Valuing our Fx Forward portfolios using different models.](#accrual_override)
* [Valuing bond PV using an externally provided market quote for the bond.](#external_bond_price)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of a bond instrument based on user provided parameters.](#bond_definition)
* [Using the StructureMarketData store to hold your OIS yield curve data in way that enables it to be discovered during the bond valuation process.](#structured_market_data)
* [Configuring recipes to run built in LUSID bond valuation models that make use of the structured data (OIS Yield Curve) you provided.](#recipe_configuration)
* [Using aggregation requests to return the accrued interest as well as the PV of the bond based on our instrument definition.](#accrued_interest)
* [Using the StructuredResultData store to override the accrued interest calculation and instead use static values.](#structured_result_data)
* [Updating our recipes to make use of the StructuredResultData entries in your valuations.](#structured_result_data)
* [Upserting bond market price as quotes and configuring recipes that value bonds by using the market quotes.](#external_bond_price)

<br>

For this notebook example we'll work the following set of GBP/USD Forwards with 2Y, 5Y and 10Y maturities:

* Trade Date : 02 Aug 2020
* Maturity Dates :
    * 02 Aug 2023 (3Y)
    * 02 Aug 2025 (5Y)
    * 02 Aug 2030 (10Y)
* Spot Rate : 1.2508
* Domestic Amount : 100,000,000
* Foreign Amounts :
    * 125,710,000 (3Y)
    * 126,485,000 (5Y)
    * 130,430,000 (10Y)


## Setup LUSID and LUSID API objects.

In [ ]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytzfrom IPython.core.display import display
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)
from lusidtools.jupyter_tools.stop_execution import StopExecution


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)

# Setup the scope we'll use in this notebook:
scope = "fx-forward-pricing-nb"

In [ ]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    return pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])


## Define our Fx Forward Instruments

We'll start by defining our forwards using LUSID's internal representation of an Fx forward instrument. For a detailed
breakdown the internal representation take a look at the "FxForwardInstrument" section of the [LUSID API Swagger Specification](https://www.lusid.com/api/swagger/index.html)

### Initialise FxForward Parameters

Let's setup the basic parameters required for our set of Fx forwards:

In [ ]:
trade_date = datetime(2020, 7, 2, tzinfo=pytz.utc)
maturity_dates = {
    '3Y' : datetime(2023, 7, 2, tzinfo=pytz.utc),
    '5Y' : datetime(2025, 7, 2, tzinfo=pytz.utc),
    '10Y': datetime(2030, 7, 2, tzinfo=pytz.utc)
}
spot_rate =  1.2508
dom_amount = 100000000
fgn_amounts = {
    '3Y' : 125710000,
    '5Y' : 126485000,
    '10Y': 130430000
}

### Create FxForwardInstrument Definitions

We now feed those parameters to construct the LUSID representation of an FxForward:

In [ ]:
def create_fx_forward_instrument_definition(dom_amount, fgn_amount, spot_rate, start_date, maturity_date):
    return models.FxForwardInstrument(
            dom_amount=dom_amount,
            fgn_amount=-fgn_amount,
            fgn_ccy="USD",
            ref_spot_rate=spot_rate,
            start_date=start_date.isoformat(),
            maturity_date=maturity_date,
            dom_ccy="GBP",
            instrument_type="FxForward")

gbpusd_fx_fwd_3y = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['3Y'], spot_rate, trade_date, maturity_dates['3Y'])
gbpusd_fx_fwd_5y = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['5Y'], spot_rate, trade_date, maturity_dates['5Y'])
gbpusd_fx_fwd_10y = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['10Y'], spot_rate, trade_date, maturity_dates['10Y'])

## Supply Market Data

Before we can price our Fx forwards we need to ensure LUSID has the required market data available. Let's upsert the
GBP/USD FX rate. When providing market data to LUSID we also need to identify the supplier of the data and the scope. The supplier
is referenced further in the notebook when we instruct LUSID how to source data for a particular aggregation.

### Upsert Spot Fx Rate

In [ ]:
market_data_scope = 'fx-forward-pricing-nb-market-data'
market_supplier = 'Lusid'

def upsert_gbp_usd_fx_rate(rate, effective_at):
    upsert_quote_request = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="GBP/USD",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    response = quotes_api.upsert_quotes(
            scope=market_data_scope,
            quotes={"gbp-usd-01": upsert_quote_request})

    if response.failed:
        raise StopExecution(f"Failed to upload yield curve:{response.failed}")

    display(f"GBP/USD @ {spot_rate} for {effective_at} uploaded to quote store.")


upsert_gbp_usd_fx_rate(spot_rate, trade_date)

## Defining our Fx Forward Valuation

With our Fx forward instruments now defined we can move onto valuing them via LUSID's aggregation engine. But before we
can execute the aggregation need to configure a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used")
instructing LUSID on how to value our Fx Forwards. Specifically we need to select the Fx Forward pricing model to use and identify
where the aggregation can resolve the market data the model needs.

### Selecting a Pricing Model

The pricing model we wish to use is passed in through defining a PricingContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters. For this notebook we'll use one of the supported Vendor models from VolMaster:

In [ ]:
def create_pricing_context():
    vendor_model_rule = models.VendorModelRule(
        supplier="VolMaster",
        model_name="VendorDefault",
        instrument_type="FxForward",
        parameters="{}")

    return models.PricingContext(
        model_rules=[vendor_model_rule]
    )

pricing_context = create_pricing_context()

### Selecting the Market Data

We can instruct LUSID on where to resolve market data required for pricing our Fx forwards through defining a MarketContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters. Recall that when we upserted our quote we passed in a supplier and scope. One of
the powerful features in LUSID is the ability to run valuations against market data from different suppliers, or even define market data retrieval rules
to fallback to suppliers should your primary supplier not have the required data.

However our example is trivial as we only require the spot rate we loaded in earlier. So we can define a simple market context :

In [ ]:
def create_market_context():
    return models.MarketContext(
                options=models.MarketOptions(
                    default_supplier=market_supplier, default_scope=market_data_scope))

market_context = create_market_context()

### Configure our FX Forward Recipe

Now that we've defined what pricing model to use and where to source the market data we can bring those instructions
together in a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used"):

In [ ]:
def create_fx_forward_pricing_recipe():
    return models.ConfigurationRecipe(
        code="Fx Forward Pricing Recipe",
        pricing=pricing_context,
        market=market_context)

fx_forward_pricing_recipe = create_fx_forward_pricing_recipe()

In [ ]:
## Price the FX Forwards

